### Data Loader and Custom Dataset
---

Dataset : MNIST
* Train Data : [Train](https://pjreddie.com/media/files/mnist_train.csv) - 60000 Training Samples
* Test : [Test](https://pjreddie.com/media/files/mnist_test.csv) - 10000 Testing Samples


### The Dataset Class
---
The Dataset class has three important class functions:
* `__init__()`: as usual, the starting point where we will initialize everything that we use in the class.
* `__len__()`: this returns the length of the dataset. Simply this will return the number of samples in the dataset.
* `__getitem__()`: this function returns a sample from the dataset when we provide an index value to it.  

### **1.Small Example Dataset Class**

In [1]:
'''
We can do amazing things with PyTorch Dataset class. We need to ensure that we are overriding two 
of it's functions, 
`__len__()`: returns the size of the dataset, that is, total number of samples.
`__getitem__()`: when given an index, returns the data sample correspoding to that index.
'''

import numpy as np
from torch.utils.data import Dataset

Define the Class ExampleDataset

In [2]:
class ExampleDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

Running the dataset

In [5]:
sample_data = np.arange(0,10)
print('the whole data:', sample_data)

dataset = ExampleDataset(sample_data)
print('Number of samples in the data:', len(dataset))

print(dataset[0])
print(dataset[0:5])

the whole data: [0 1 2 3 4 5 6 7 8 9]
Number of samples in the data: 10
0
[0 1 2 3 4]


### **Creating Datasets and DataLoaders from CSV Files**


### **1. Load Libraries**

In [27]:
# torch loaded...!!!
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# torchvision loaded...!!!
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchsummary import summary

# other Libraries loaded...!!!
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

### **2.GPU Device Loaded**

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### **3.Load and Prepare the Data**

In [50]:
# load the dataset
train = pd.read_csv('https://pjreddie.com/media/files/mnist_train.csv')
test = pd.read_csv('https://pjreddie.com/media/files/mnist_test.csv')

In [51]:
type(train)

pandas.core.frame.DataFrame

In [52]:
# get the image pixels and labels
train_images = train.iloc[:,1:]
train_labels = train.iloc[:,0]

test_images = test.iloc[:,1:]
test_labels = test.iloc[:,0]

In [53]:
print(train_images.shape, train_labels.shape)
print(test_labels.shape, test_labels.shape)

(59999, 784) (59999,)
(9999,) (9999,)


### **4.Define the Transforms**

In [54]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

### **5.Prepare the Custom Dataset**

In [55]:
# Custom class
class MNISTDataset(Dataset):
    def __init__(self, images, labels = None, transforms = None):
        self.X = images
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        data = self.X.iloc[index, :]
        data = np.asarray(data).astype(np.uint8).reshape(28,28,1)
        
        if self.transforms:
            data = self.transforms(data)
        
        if self.y is not None:
            return data, self.y[index]
        else:
            return data

### **6. Dataset Reading and Load the dataset.**

In [56]:
# Create the dataset
trainData = MNISTDataset(train_images, train_labels, transform)
testData = MNISTDataset(test_images, test_labels, transform)

# Data Loader
trainLoader = DataLoader(trainData, batch_size = 128, shuffle=True)
testLoader = DataLoader(testData, batch_size = 128, shuffle=True)

* In the `__init__()` function we initialize the images, labels, and transforms. Note that by default the labels and transforms parameters are None. We will pass them as arguments depending on our requirements for the project. 

* The `__len__()` function returns the length as usual.

* Most of the work is being done in the `__getitem__()` function. So, we get the data on the index by index basis. For each index, we get the pixel data for the entire row. then we convert the data to Numpy array and reshape it into 28×28 gray-scale images. then Apply the transforms to the pixel data based on the transforms that we have defined earlier. Also, we return the pixel data along with the corresponding label.

* The next few lines of code are fairly straightforward. We create two objects, trainData and testData for the MNISTDataset() class. We pass the image pixels, the image labels, and the transforms as arguments. we define the `trainLoader` and `testLoader`. You must find this line very similar to the directly getting the dataloader from the PyTorch MNIST dataset.

### **7.Define the Neural Network, Train, and Test it**

In [57]:
# define the neural net class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=800, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [58]:
summary(net, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 20, 24, 24]          520
├─Conv2d: 1-2                            [-1, 50, 8, 8]            25,050
├─Linear: 1-3                            [-1, 500]                 400,500
├─Linear: 1-4                            [-1, 10]                  5,010
Total params: 431,080
Trainable params: 431,080
Non-trainable params: 0
Total mult-adds (M): 2.29
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.64
Estimated Total Size (MB): 1.76


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 20, 24, 24]          520
├─Conv2d: 1-2                            [-1, 50, 8, 8]            25,050
├─Linear: 1-3                            [-1, 500]                 400,500
├─Linear: 1-4                            [-1, 10]                  5,010
Total params: 431,080
Trainable params: 431,080
Non-trainable params: 0
Total mult-adds (M): 2.29
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.64
Estimated Total Size (MB): 1.76

### **8. Calculate the Loss Function and Optimization**

In [59]:
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### **9.training and test function and execute the functions**

In [64]:
def train(net, trainloader):
    for epoch in range(10): # no. of epochs
        running_loss = 0
        with tqdm(trainLoader, unit="batch") as tepoch:
            for data in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}")
                # data pixels and labels to GPU if available
                inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
                # set the parameter gradients to zero
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                # propagate the loss backward
                loss.backward()
                # update the gradients
                optimizer.step()
                tepoch.set_postfix(loss=loss)
                running_loss += loss.item()
                
            print('[Epoch %d] loss: %.3f' %(epoch + 1, running_loss/len(trainloader)))
 
    print('Done Training')
    
    
def test(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testLoader:
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on test images: %0.3f %%' % (100 * correct / total))
    
train(net, trainLoader)
test(net, testLoader)

Epoch 1: 100%|██████████| 469/469 [00:35<00:00, 13.31batch/s, loss=tensor(0.3137, device='cuda:0', grad_fn=<NllLossBackward>)]


[Epoch 1] loss: 0.917


Epoch 2: 100%|██████████| 469/469 [00:29<00:00, 15.92batch/s, loss=tensor(0.1925, device='cuda:0', grad_fn=<NllLossBackward>)]


[Epoch 2] loss: 0.249


Epoch 3: 100%|██████████| 469/469 [00:28<00:00, 16.47batch/s, loss=tensor(0.1427, device='cuda:0', grad_fn=<NllLossBackward>)]


...

[Epoch 8] loss: 0.076


Epoch 9: 100%|██████████| 469/469 [00:23<00:00, 19.57batch/s, loss=tensor(0.0840, device='cuda:0', grad_fn=<NllLossBackward>)]


[Epoch 9] loss: 0.070


Epoch 10: 100%|██████████| 469/469 [00:26<00:00, 17.81batch/s, loss=tensor(0.0777, device='cuda:0', grad_fn=<NllLossBackward>)]


[Epoch 10] loss: 0.065
Done Training
Accuracy of the network on test images: 98.220 %
